In [24]:
import pandas as pd
import re

data = pd.read_excel("idealista_gijon_centro.xlsx")

In [33]:
# 2. Definir la función de extracción
def extraer_info(titulo):
    # Valores por defecto
    tipo = None
    direccion = None
    numero = None
    ubicacion = None
    ciudad = 'Gijón'  # Fijo según tu petición

    try:
        if not isinstance(titulo, str):
            return pd.Series([None]*5, index=['Tipo', 'Dirección', 'Número', 'Ubicación', 'Ciudad'])

        # -- A. Extraer TIPO (lo que hay antes de " en ") --
        parts_en = titulo.split(' en ', 1)
        tipo = parts_en[0].strip()
        
        # Si no hay parte después de "en", terminamos aquí
        if len(parts_en) < 2:
            return pd.Series([tipo, direccion, numero, ubicacion, ciudad], 
                             index=['Tipo', 'Dirección', 'Número', 'Ubicación', 'Ciudad'])

        resto = parts_en[1]

        # -- B. Procesar el resto separando por comas --
        # Estructura esperada: [Dirección...], [Ubicación], [Ciudad]
        geo_parts = [p.strip() for p in resto.split(',')]

        # La ciudad suele ser el último elemento, pero lo forzamos a Gijón como pediste.
        # Analizamos los elementos anteriores para encontrar Ubicación y Dirección.

        if len(geo_parts) >= 2:
            # Asumimos que el penúltimo elemento es la UBICACIÓN (Barrio/Zona)
            # Ejemplo: "..., Centro - Puerto, Gijón" -> Ubicación = "Centro - Puerto"
            ubicacion = geo_parts[-2]

            # Todo lo que queda antes son partes de la DIRECCIÓN
            address_parts = geo_parts[:-2]

            if address_parts:
                # -- C. Extraer NÚMERO si existe --
                # Comprobamos si la última parte de la dirección parece un número (dígitos)
                posible_numero = address_parts[-1]
                
                # Regex simple: dígitos solos o dígitos seguidos de una letra (ej: 20, 20A)
                if re.match(r'^\d+[a-zA-Z]?$', posible_numero):
                    numero = posible_numero
                    # La dirección es todo lo anterior menos el número
                    direccion = ", ".join(address_parts[:-1])
                else:
                    # Si no parece un número, toda esta parte es la dirección
                    direccion = ", ".join(address_parts)
            else:
                # Caso: "Piso en Centro - Puerto, Gijón" (Sin calle específica)
                direccion = None 

        elif len(geo_parts) == 1:
            # Caso raro: "Piso en Tremañes" (donde Tremañes es ubicación)
            # Si solo hay uno y no es Gijón, asumimos que es la ubicación
            if geo_parts[0] != 'Gijón':
                ubicacion = geo_parts[0]

    except Exception as e:
        print(f"Error procesando: {titulo} -> {e}")

    return pd.Series([tipo, direccion, numero, ubicacion, ciudad], 
                     index=['Tipo', 'Dirección', 'Número', 'Ubicación', 'Ciudad'])

def obtener_planta(texto):
    # Si el valor es nulo/vacío
    if pd.isna(texto):
        return None
    
    # Convertimos a minúsculas para facilitar la búsqueda
    texto_str = str(texto).lower()
    
    # 1. Caso "Bajo" -> Asignamos 0
    if 'bajo' in texto_str:
        return 0
    
    # 2. Caso "Entreplanta" -> Devolvemos el texto o un código (ej. 0.5 o -1)
    if 'entreplanta' in texto_str:
        return 'Entreplanta'
    
    # 3. Caso estándar "Planta X"
    # Buscamos la palabra "planta" seguida de espacios y uno o más dígitos
    match = re.search(r'planta\s+(\d+)', texto_str)
    if match:
        return int(match.group(1))
    
    # Si no encuentra nada conocido
    return None


In [35]:
# 3. Aplicar la función al DataFrame
nuevas_columnas = data['Título'].apply(extraer_info)

# 4. Unir con el DataFrame original
df_final = pd.concat([data, nuevas_columnas], axis=1)

# Aplicar la función a la columna
df_final["Planta"] = df_final['Planta_Descripcion'].apply(obtener_planta)

In [37]:
df_final.drop(columns=["Título", 'Planta_Descripcion']).to_excel('idealista_gijon_2025.xlsx', index=False)